# RAGit

### Let's create a function load repo

In [2]:
import os
import shutil
from git import Repo

In [3]:
def clone_repo(repo_url, clone_dir="repo_temp"):
    try:
        if os.path.exists(clone_dir):
            shutil.rmtree(clone_dir)
        print("Cloning repositry...")
        Repo.clone_from(repo_url, clone_dir)
        print(f"✅ Repo cloned to: {clone_dir}")
        return clone_dir
    except GitCommandError as e:
        print(f"❌ Git error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

In [4]:
repo_url = "https://github.com/Sohan-Choudhary05/RAGit"
repo_temp = r"C:\Users\Ram Choudhary\OneDrive\Desktop\RAGit\rpo_tmp"
clone_repo(repo_url, repo_temp)

Cloning repositry...
✅ Repo cloned to: C:\Users\Ram Choudhary\OneDrive\Desktop\RAGit\rpo_tmp


'C:\\Users\\Ram Choudhary\\OneDrive\\Desktop\\RAGit\\rpo_tmp'